# Maximum Flow Algorithmen
Mit Flussgraphen lassen sich alle Probleme modellieren, bei denen ein Fluss von einer Quelle zu einer Senke stattfindet. Dazu zählen beispielsweise Rohrleitungen und Kabel, aber auch Betrachtungen des Straßenverkehrs.

In [ ]:
from tui_dsmt.graph import MaximumFlow, FordFulkerson
from graph_data import G, G2

## Inhaltsverzeichnis
- [Flussgraphen](#Flussgraphen)
- [Ford-Fulkerson-Algorithmus](#Ford-Fulkerson-Algorithmus)
- [Komplexität durch Pfadwahl](#Komplexität-durch-Pfadwahl)

## Flussgraphen
Ein **Flussgraph** - häufig auch als *Netzwerk* bezeichnet - ist ein <u>gerichteter</u> Graph $G = (V, E)$ mit zwei besonderen Knoten. $s$ bezeichnet eine Quelle (**S**ource) und $t$ eine Senke (**T**arget). Zusätzlich existiert eine Funktion $u : E \rightarrow \mathbb{R}_+$, die jeder Kante eine nichtnegative **Kapazität** zuweist.

Ein **Fluss** ist eine Funktion $f : E \rightarrow \mathbb{R}_+$, die jeder Kante einen nichtnegativen Fluss zuweist. Für den Fluss müssen zwei Bedingungen gelten:
- Der Fluss muss kleiner oder gleich der Kapazität sein: $\forall e \in E : f(e) \leq u(e)$
- Abgesehen von der Quelle und der Senke muss in jeden Knoten so viel hinein- wie hinausfließen.

Im Beispiel der Rohrleitungen könnte es sich bei den Knoten um Pumpen handeln. Währenddessen beschreibt die Kapazität den maximalen Durchfluss und der Fluss den aktuellen Durchfluss. Fluss und Kapazität werden in grafischen Darstellungen in der Regel durch einen Schrägstrich getrennt neben der jeweiligen Kante notiert.

In [ ]:
MaximumFlow(G)

## Ford-Fulkerson-Algorithmus
Der Algorithmus von Ford und Fulkerson versucht Pfade von der Quelle zur Senke und währenddessen den Fluss kontinuierlich zu erhöhen. Es sind jedoch noch weitere Konzepte notwendig, um den Algorithmus zu verstehen:
- Der **Residualgraph** besitzt die gleichen Knoten- und Kantenmengen, wird aber zusätzlich um Rückkanten erweitert. (Für jede gerichtete Kante wird eine Kante mit vertauschten Start- und Zielknoten eingefügt.)
- Ein **augmentierender Pfad** verbindet Quelle und Senke und besitzt freie Kapazitäten zur Erhöhung des Flusses.
- Ein **Flaschenhals** ist die Kante entlang eines Pfades, welche die geringste verbleibende Kapazität besitzt.

Der Algorithmus arbeitet dann wie folgt:
1. Aus dem Graphen wird der Residualgraph gebildet.
2. Jeder Kante wird der Fluss $0$ zugeordnet.
3. Solange ein Pfad von $s$ nach $t$ **im Residualgraphen** gefunden werden kann
    1. ist der Wert des Flaschenhalses zu bestimmen,
    2. für alle Kanten im Pfad zum Fluss der Wert des Flaschenhalses zu addieren und
    3. für alle Gegenkanten des Pfades vom Fluss der Wert des Flaschenhalses zu subtrahieren.

Für rationale Kapazitäten endet der Algorithmus in endlicher Zeit.

In [ ]:
FordFulkerson(G)

## Komplexität durch Pfadwahl
Sind alle Kapazitäten rational endet der Algorithmus nach endlich vielen Schritten. Die Komplexität hängt aber maßgeblich von der Wahl des Pfades ab. So lassen sich leicht Graphen konstruieren, die trotz geringer Größe eine große Anzahl an Iterationen benötigen.

In [ ]:
FordFulkerson(G2)

Abhilfe kann zum Beispiel geschaffen werden, indem in jedem Schritt der kürzeste augmentierende Pfad gewählt wird.